<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/224-LangChain-ChatBot-H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install openai
!pip install gradio
!pip install langchain langchain_community
!pip install python-dotenv

In [ ]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [1]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


감정 분석 기능:
유저의 메시지에 대한 감정 분석을 수행하여, 챗봇의 응답을 감정에 따라 조절할 수 있습니다.

In [3]:
from textblob import TextBlob
import gradio as gr
from langchain.schema import HumanMessage, AIMessage

def predict(message, history):
    if history is None:
        history = []

    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    # Adding only the current human message to the format history
    history_langchain_format.append(HumanMessage(content=message))

    # Generate GPT response
    gpt_response = llm(history_langchain_format)

    # Analyzing the sentiment of the human message
    human_sentiment = TextBlob(message).sentiment
    if human_sentiment.polarity < 0:
        gpt_response.content = "I'm sorry to hear that. " + gpt_response.content

    # Analyzing the sentiment of the AI response
    ai_sentiment = TextBlob(gpt_response.content).sentiment
    if ai_sentiment.polarity < 0:
        gpt_response.content = gpt_response.content + " I hope this helps."

    # Append the human message and AI response to history
    history.append((message, gpt_response.content))

    return gpt_response.content, history

# Define Gradio interface
iface = gr.Interface(
    fn=predict,
    inputs=["text", "state"],
    outputs=["text", "state"],
    live=False  # Turn off streaming
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5a9cb7b6c766aaf9a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
